In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Psoriasis"
cohort = "GSE252029"

# Input paths
in_trait_dir = "../../input/GEO/Psoriasis"
in_cohort_dir = "../../input/GEO/Psoriasis/GSE252029"

# Output paths
out_data_file = "../../output/preprocess/Psoriasis/GSE252029.csv"
out_gene_data_file = "../../output/preprocess/Psoriasis/gene_data/GSE252029.csv"
out_clinical_data_file = "../../output/preprocess/Psoriasis/clinical_data/GSE252029.csv"
json_path = "../../output/preprocess/Psoriasis/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Guselkumab reduces disease- and mechanism-related biomarkers more than adalimumab in patients with psoriasis: a VOYAGE 1 substudy"
!Series_summary	"In the phase 3 VOYAGE-1 trial (ClinicalTrials.gov identifier: NCT02207231), guselkumab demonstrated superior efficacy versus placebo and the tumor necrosis factor (TNF)-α antagonist, adalimumab, in patients with moderate-to-severe plaque psoriasis (Blauvelt et al., 2017). Here, skin samples were collected from patients in VOYAGE-1 and pharmacodynamic (PD) responses to guselkumab (vs adalimumab) treatment were assessed over 48 weeks."
!Series_overall_design	"Psoriasis lesional skin (LS) and nonlesional skin (NL) samples were collected from 39 VOYAGE-1 trial patients who provided consent to participate in an optional skin biopsy substudy to evaluate PD effects on psoriasis transcriptomics, and were profiled using GeneChip HT HG-U133+ PM 96-Array Plate (Affymetrix, Santa Clara, CA, USA)"
Sample Characteri

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# 1. Gene Expression Data Availability
# From the background information, we can see that this dataset contains gene expression data
# using GeneChip HT HG-U133+ PM 96-Array Plate (Affymetrix)
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# Looking at the sample characteristics dictionary:

# 2.1 Trait (Psoriasis)
# From the dictionary, we can see this is a psoriasis dataset
# The skin type (LS = Lesional Skin, NL = Nonlesional Skin) at key 4 indicates psoriasis status
trait_row = 4

def convert_trait(value):
    """Convert skin type to binary trait status (Psoriasis)"""
    if pd.isna(value):
        return None
    
    # Extract the value after the colon
    if ":" in value:
        value = value.split(":", 1)[1].strip()
    
    # LS (Lesional Skin) indicates psoriasis is present (1)
    # NL (Nonlesional Skin) indicates psoriasis is not present (0)
    if value.upper() == "LS":
        return 1
    elif value.upper() == "NL":
        return 0
    else:
        return None

# 2.2 Age
# There is no age information in the sample characteristics dictionary
age_row = None

def convert_age(value):
    """Convert age to continuous value"""
    # This function won't be used but needs to be defined
    if pd.isna(value):
        return None
    
    if ":" in value:
        value = value.split(":", 1)[1].strip()
    
    try:
        return float(value)
    except:
        return None

# 2.3 Gender
# There is no gender information in the sample characteristics dictionary
gender_row = None

def convert_gender(value):
    """Convert gender to binary value"""
    # This function won't be used but needs to be defined
    if pd.isna(value):
        return None
    
    if ":" in value:
        value = value.split(":", 1)[1].strip()
    
    value = value.lower()
    if value in ["female", "f"]:
        return 0
    elif value in ["male", "m"]:
        return 1
    else:
        return None

# 3. Save Metadata
# Check if trait data is available
is_trait_available = trait_row is not None

# Validate and save cohort info (initial filtering)
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
# Since trait_row is not None, we need to extract clinical features
if trait_row is not None:
    clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the clinical data
    preview = preview_df(clinical_df)
    print("Clinical data preview:")
    print(preview)
    
    # Save the clinical data to CSV
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    clinical_df.to_csv(out_clinical_data_file, index=False)
    print(f"Clinical data saved to {out_clinical_data_file}")


Clinical data preview:
{'GSM7992374': [1.0], 'GSM7992375': [0.0], 'GSM7992376': [1.0], 'GSM7992377': [0.0], 'GSM7992378': [1.0], 'GSM7992379': [0.0], 'GSM7992380': [1.0], 'GSM7992381': [0.0], 'GSM7992382': [1.0], 'GSM7992383': [0.0], 'GSM7992384': [1.0], 'GSM7992385': [0.0], 'GSM7992386': [1.0], 'GSM7992387': [0.0], 'GSM7992388': [1.0], 'GSM7992389': [1.0], 'GSM7992390': [0.0], 'GSM7992391': [1.0], 'GSM7992392': [1.0], 'GSM7992393': [0.0], 'GSM7992394': [1.0], 'GSM7992395': [1.0], 'GSM7992396': [1.0], 'GSM7992397': [1.0], 'GSM7992398': [1.0], 'GSM7992399': [1.0], 'GSM7992400': [1.0], 'GSM7992401': [1.0], 'GSM7992402': [1.0], 'GSM7992403': [1.0], 'GSM7992404': [1.0], 'GSM7992405': [1.0], 'GSM7992406': [1.0], 'GSM7992407': [1.0], 'GSM7992408': [1.0], 'GSM7992409': [1.0], 'GSM7992410': [1.0], 'GSM7992411': [1.0], 'GSM7992412': [1.0], 'GSM7992413': [1.0], 'GSM7992414': [1.0], 'GSM7992415': [1.0], 'GSM7992416': [1.0], 'GSM7992417': [1.0], 'GSM7992418': [1.0], 'GSM7992419': [1.0], 'GSM799242

### Step 3: Gene Data Extraction

In [4]:
# 1. Re-identify the SOFT and matrix files to ensure we have the correct paths
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Extract the gene expression data from the matrix file
gene_data = get_genetic_data(matrix_file)

# 3. Print the first 20 row IDs (gene or probe identifiers)
print("\nFirst 20 gene/probe identifiers:")
print(gene_data.index[:20])

# 4. Print the dimensions of the gene expression data
print(f"\nGene data dimensions: {gene_data.shape[0]} genes × {gene_data.shape[1]} samples")

# Note: we keep is_gene_available as True since we successfully extracted gene expression data
is_gene_available = True



First 20 gene/probe identifiers:
Index(['1007_PM_s_at', '1053_PM_at', '117_PM_at', '121_PM_at', '1255_PM_g_at',
       '1294_PM_at', '1316_PM_at', '1320_PM_at', '1405_PM_i_at', '1431_PM_at',
       '1438_PM_at', '1487_PM_at', '1494_PM_f_at', '1552256_PM_a_at',
       '1552257_PM_a_at', '1552258_PM_at', '1552261_PM_at', '1552263_PM_at',
       '1552264_PM_a_at', '1552266_PM_at'],
      dtype='object', name='ID')

Gene data dimensions: 54715 genes × 181 samples


### Step 4: Gene Identifier Review

In [5]:
# These identifiers appear to be Affymetrix probe IDs (like '1007_PM_s_at') from an Affymetrix microarray platform
# They are not standard human gene symbols and will need to be mapped to proper gene symbols

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. First get the file paths using geo_get_relevant_filepaths function
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Extract gene annotation data from the SOFT file
gene_annotation = get_gene_annotation(soft_file)

# 3. Preview the gene annotation dataframe
print("Gene annotation dataframe column names:")
print(gene_annotation.columns)

# Preview the first few rows to understand the data structure
print("\nPreview of gene annotation data:")
annotation_preview = preview_df(gene_annotation, n=3)
print(annotation_preview)

# Maintain gene availability status as True based on previous steps
is_gene_available = True


Gene annotation dataframe column names:
Index(['ID', 'GB_ACC', 'SPOT_ID', 'Species Scientific Name', 'Annotation Date',
       'Sequence Type', 'Sequence Source', 'Target Description',
       'Representative Public ID', 'Gene Title', 'Gene Symbol',
       'ENTREZ_GENE_ID', 'RefSeq Transcript ID',
       'Gene Ontology Biological Process', 'Gene Ontology Cellular Component',
       'Gene Ontology Molecular Function'],
      dtype='object')

Preview of gene annotation data:
{'ID': ['1007_PM_s_at', '1053_PM_at', '117_PM_at'], 'GB_ACC': ['U48705', 'M87338', 'X51757'], 'SPOT_ID': [nan, nan, nan], 'Species Scientific Name': ['Homo sapiens', 'Homo sapiens', 'Homo sapiens'], 'Annotation Date': ['Aug 20, 2010', 'Aug 20, 2010', 'Aug 20, 2010'], 'Sequence Type': ['Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence'], 'Sequence Source': ['Affymetrix Proprietary Database', 'GenBank', 'Affymetrix Proprietary Database'], 'Target Description': ['U48705 /FEATURE=mRNA /DEFINITION=HSU48705 Human

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Determine which columns contain the probe IDs and gene symbols
# Looking at the previews, 'ID' column in gene_annotation matches the indices in gene_data (probe IDs)
# 'Gene Symbol' contains the corresponding gene symbols

# 2. Extract gene mapping from the annotation
mapping_df = get_gene_mapping(
    annotation=gene_annotation,
    prob_col='ID',
    gene_col='Gene Symbol'
)

# 3. Apply the gene mapping to convert probes to gene expressions
gene_data = apply_gene_mapping(expression_df=gene_data, mapping_df=mapping_df)

# 4. Check the dimensionality change after mapping
print(f"\nGene data dimensions after mapping: {gene_data.shape[0]} genes × {gene_data.shape[1]} samples")

# 5. Print a sample of the first few gene symbols after mapping
print("\nSample of gene symbols after mapping:")
print(gene_data.index[:10])



Gene data dimensions after mapping: 18989 genes × 181 samples

Sample of gene symbols after mapping:
Index(['A1BG', 'A1CF', 'A2BP1', 'A2LD1', 'A2M', 'A2ML1', 'A4GALT', 'A4GNT',
       'AAA1', 'AAAS'],
      dtype='object', name='Gene')


### Step 7: Data Normalization and Linking

In [8]:
# 1. Check if gene data is available after mapping
if gene_data.shape[0] == 0:
    print("Error: Gene expression matrix is empty after mapping.")
    # Mark the dataset as not usable due to lack of gene expression data
    is_usable = validate_and_save_cohort_info(
        is_final=True,
        cohort=cohort,
        info_path=json_path,
        is_gene_available=False,  # No usable gene data
        is_trait_available=True,
        is_biased=True,
        df=pd.DataFrame(),
        note="Failed to map probe IDs to gene symbols. The annotation format may not be compatible with the extraction methods."
    )
    print("Dataset deemed not usable due to lack of gene expression data.")
else:
    # Only proceed with normalization if we have gene data
    print("Normalizing gene symbols...")
    gene_data_normalized = normalize_gene_symbols_in_index(gene_data)
    print(f"Gene data shape after normalization: {gene_data_normalized.shape[0]} genes × {gene_data_normalized.shape[1]} samples")

    # Save the normalized gene data
    os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
    gene_data_normalized.to_csv(out_gene_data_file)
    print(f"Normalized gene expression data saved to {out_gene_data_file}")
    
    # Extract clinical features from the original data source
    print("Extracting clinical features from the original source...")
    # Get background information and clinical data again
    background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
    clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
    background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)
    
    # Extract clinical features
    selected_clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    print("Extracted clinical features preview:")
    print(preview_df(selected_clinical_df))
    print(f"Clinical data shape: {selected_clinical_df.shape}")
    
    # Save the extracted clinical features
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    selected_clinical_df.to_csv(out_clinical_data_file)
    print(f"Clinical features saved to {out_clinical_data_file}")
    
    # Link clinical and genetic data
    print("Linking clinical and genetic data...")
    linked_data = geo_link_clinical_genetic_data(selected_clinical_df, gene_data_normalized)
    print(f"Linked data shape: {linked_data.shape}")
    
    # Check if the linked data has adequate data
    if linked_data.shape[0] == 0 or linked_data.shape[1] <= 4:  # 4 is an arbitrary small number
        print("Error: Linked data has insufficient samples or features.")
        is_usable = validate_and_save_cohort_info(
            is_final=True,
            cohort=cohort,
            info_path=json_path,
            is_gene_available=True,
            is_trait_available=True,
            is_biased=True,
            df=linked_data,
            note="Failed to properly link gene expression data with clinical features."
        )
        print("Dataset deemed not usable due to linking failure.")
    else:
        # Handle missing values systematically
        print("Handling missing values...")
        linked_data_clean = handle_missing_values(linked_data, trait_col=trait)
        print(f"Data shape after handling missing values: {linked_data_clean.shape}")
        
        # Check if there are still samples after missing value handling
        if linked_data_clean.shape[0] == 0:
            print("Error: No samples remain after handling missing values.")
            is_usable = validate_and_save_cohort_info(
                is_final=True,
                cohort=cohort,
                info_path=json_path,
                is_gene_available=True,
                is_trait_available=True,
                is_biased=True,
                df=pd.DataFrame(),
                note="All samples were removed during missing value handling."
            )
            print("Dataset deemed not usable as all samples were filtered out.")
        else:
            # Check if the dataset is biased
            print("\nChecking for bias in feature variables:")
            is_biased, linked_data_final = judge_and_remove_biased_features(linked_data_clean, trait)
            
            # Conduct final quality validation
            is_usable = validate_and_save_cohort_info(
                is_final=True,
                cohort=cohort,
                info_path=json_path,
                is_gene_available=True,
                is_trait_available=True,
                is_biased=is_biased,
                df=linked_data_final,
                note="Dataset contains gene expression data for Crohn's Disease patients, examining response to Infliximab treatment."
            )
            
            # Save linked data if usable
            if is_usable:
                os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
                linked_data_final.to_csv(out_data_file)
                print(f"Linked data saved to {out_data_file}")
                print(f"Final dataset shape: {linked_data_final.shape}")
            else:
                print("Dataset deemed not usable for trait association studies, linked data not saved.")

Normalizing gene symbols...
Gene data shape after normalization: 18622 genes × 181 samples


Normalized gene expression data saved to ../../output/preprocess/Psoriasis/gene_data/GSE252029.csv
Extracting clinical features from the original source...


Extracted clinical features preview:
{'GSM7992374': [1.0], 'GSM7992375': [0.0], 'GSM7992376': [1.0], 'GSM7992377': [0.0], 'GSM7992378': [1.0], 'GSM7992379': [0.0], 'GSM7992380': [1.0], 'GSM7992381': [0.0], 'GSM7992382': [1.0], 'GSM7992383': [0.0], 'GSM7992384': [1.0], 'GSM7992385': [0.0], 'GSM7992386': [1.0], 'GSM7992387': [0.0], 'GSM7992388': [1.0], 'GSM7992389': [1.0], 'GSM7992390': [0.0], 'GSM7992391': [1.0], 'GSM7992392': [1.0], 'GSM7992393': [0.0], 'GSM7992394': [1.0], 'GSM7992395': [1.0], 'GSM7992396': [1.0], 'GSM7992397': [1.0], 'GSM7992398': [1.0], 'GSM7992399': [1.0], 'GSM7992400': [1.0], 'GSM7992401': [1.0], 'GSM7992402': [1.0], 'GSM7992403': [1.0], 'GSM7992404': [1.0], 'GSM7992405': [1.0], 'GSM7992406': [1.0], 'GSM7992407': [1.0], 'GSM7992408': [1.0], 'GSM7992409': [1.0], 'GSM7992410': [1.0], 'GSM7992411': [1.0], 'GSM7992412': [1.0], 'GSM7992413': [1.0], 'GSM7992414': [1.0], 'GSM7992415': [1.0], 'GSM7992416': [1.0], 'GSM7992417': [1.0], 'GSM7992418': [1.0], 'GSM7992419': [1.

Data shape after handling missing values: (181, 18623)

Checking for bias in feature variables:
For the feature 'Psoriasis', the least common label is '0.0' with 38 occurrences. This represents 20.99% of the dataset.
The distribution of the feature 'Psoriasis' in this dataset is fine.



Linked data saved to ../../output/preprocess/Psoriasis/GSE252029.csv
Final dataset shape: (181, 18623)
